In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from konlpy.tag import Twitter
import os
from gensim import corpora, models, similarities
import numpy as np

station = pd.read_excel("2호선 역 목록.xlsx")
# 역 이름에 # 붙여서 리스트로 저장
station = list(station['총'])
stations = [x for x in station if str(x) != 'nan']
stations

w2v_list = []
# 포스팅 분리
for station in stations:
    try:
        doc = open('crawl/'+station + '.csv', 'r')
        data = doc.read()
        doc.close()
        phrase = data.split('\n')
    except:
        doc = open('crawl/'+station  + '.csv', 'r', encoding='utf8')
        data = doc.read()
        doc.close()
        phrase = data.split('\n')
        continue

    # 포스팅에서 한글 외 문자 제거
    import re

    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')

    # 형태소 문장 단위 추출
    dic_Twitter = []
    for i in range(len(phrase)):
        p = hangul.sub(' ', phrase[i])
        s = Twitter().pos(p, stem=True, norm=True)
        dic_Twitter.append(s)

    # 총 형태소
    name = doc.name
    dic_ad_noun=[]
    for twit in dic_Twitter:
        c = [x[0] for x in twit if x[1] =="Adjective" or x[1] == "Noun" and len(x[0]) >1 and x[0] not in ["스타","그램",name[:-4],name[:-5],"이다","있다","없다","같다","아니다","그렇다","이렇다","많다","어떻다",
                                             "스럽다","저렇다","인스타그램","맞팔","선팔","이벤트","응모"]]
        dic_ad_noun.append(c)
    x=np.str(station)
    y= models.Word2Vec(dic_ad_noun, size=100, min_count=1)
    w2v={x:y}
    w2v_list.append(w2v)
    print(station)
print("1차 준비 완료")

시청역
을지로입구역
을지로3가역
을지로4가역
동대문역사문화공원역
신당역
상왕십리역
왕십리역
한양대역
뚝섬역
성수역
건대입구역
구의역
강변역
잠실나루역
잠실역
신천역
종합운동장역
삼성역
선릉역
역삼역


In [ ]:
ps_dic=[]
for station in stations:
    file = pd.read_excel('역별 tf-idf.xlsx')
    result = pd.DataFrame(np.zeros(20))
    words = list(file.ix[:, station])
    new_words = [x for x in words if str(x) != 'nan']
    ps_dic.append({station:new_words})

In [ ]:
#w2v을 역별로 형용사 15씩 적용하기
w2v_ps_dic=[]
for i, sta in enumerate(stations):
    try:
        kk=w2v_list[i][sta]
        w2v_dic =[]
        for a in ps_dic[i][stations[i]]:
            wps={a:kk.similar_by_word(a,topn=10)}
            w2v_dic.append(wps)
        w2v_ps_dic.append((sta,w2v_dic))
    except:
        continue

In [ ]:
for v in range(len(stations)):
    x= pd.DataFrame(w2v_ps_dic[v][1][0])
    print(x)
    for i in range(len(w2v_ps_dic[v][1])):
        x= pd.concat([x,pd.DataFrame(w2v_ps_dic[v][1][i+1])], axis=1)
    x.to_excel(stations[v]+'_tf-idf_to_w2v.xlsx')

In [ ]:
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec

In [ ]:
model = Doc2Vec(min_count=1, window=10, size=100, sample=1e-4, negative=5, workers=8)

In [ ]:
for v in range(50):
    x= pd.DataFrame(w2v_ps_dic[v][1][0])
    print(x)
    for i in range(14):
        print(i+1)
        x= pd.concat([x,pd.DataFrame(w2v_ps_dic[v][1][i+1])], axis=1)
        x.to_excel(new_station_list[v]+'.xlsx')